In [134]:
import pandas as pd
import numpy as np

In [135]:
test = pd.read_csv('test.csv', encoding = 'cp949')
train = pd.read_csv('train.csv', encoding = 'cp949')
sub = pd.read_csv('sample_submission.csv', encoding = 'cp949')

In [136]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122400 entries, 0 to 122399
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   num         122400 non-null  int64  
 1   date_time   122400 non-null  object 
 2   전력사용량(kWh)  122400 non-null  float64
 3   기온(°C)      122400 non-null  float64
 4   풍속(m/s)     122400 non-null  float64
 5   습도(%)       122400 non-null  float64
 6   강수량(mm)     122400 non-null  float64
 7   일조(hr)      122400 non-null  float64
 8   비전기냉방설비운영   122400 non-null  float64
 9   태양광보유       122400 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 9.3+ MB


- target
    - 전력사용량(kWh)
- feature
    - numeric
        - 기온
        - 풍속
        - 습도
        - 강수량
        - 일조
        - 비전기냉방시설운영 : 0,1 factor
        - 태양광보유 : 0,1 factor
        - num : 건물번호
    - datetime
        - date_time

In [137]:
df_train = train.copy()
df_test = test.copy()

- date time 선언

In [138]:
df_train['date_time'] = pd.to_datetime(df_train['date_time'])
df_test['date_time'] = pd.to_datetime(df_test['date_time'])

df_train['year'] = df_train['date_time'].dt.year
df_train['month'] = df_train['date_time'].dt.month
df_train['day'] = df_train['date_time'].dt.day
df_train['hour'] = df_train['date_time'].dt.hour

df_test['year'] = df_test['date_time'].dt.year
df_test['month'] = df_test['date_time'].dt.month
df_test['day'] = df_test['date_time'].dt.day
df_test['hour'] = df_test['date_time'].dt.hour


In [139]:
df_train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,year,month,day,hour
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,2020,6,1,0
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,2020,6,1,1
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2020,6,1,2
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,2020,6,1,3
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,2020,6,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19:00:00,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0,2020,8,24,19
122396,60,2020-08-24 20:00:00,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0,2020,8,24,20
122397,60,2020-08-24 21:00:00,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0,2020,8,24,21
122398,60,2020-08-24 22:00:00,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0,2020,8,24,22


- numeric feature standardization

- get_dummies

In [140]:
# df_train = pd.concat([df_train,pd.get_dummies(df_train['num'])], axis = 1)
# df_test = pd.concat([df_test,pd.get_dummies(df_test['num'])], axis = 1)
df_test[['비전기냉방설비운영','태양광보유']] = df_test[['비전기냉방설비운영','태양광보유']].fillna(0)

cols = ['num','비전기냉방설비운영','태양광보유']

df_train[cols[1]][df_train[cols[1]] == 1] = '냉방설비 운영'
df_train[cols[1]][df_train[cols[1]] == 0] = '냉방설비 비운영' 
df_train[cols[-1]][df_train[cols[-1]] == 1] = '태양광 보유'
df_train[cols[-1]][df_train[cols[-1]] == 0] = '태양광 미보유'

df_test[cols[1]][df_test[cols[1]] == 1] = '냉방설비 운영'
df_test[cols[1]][df_test[cols[1]] == 0] = '냉방설비 비운영' 
df_test[cols[-1]][df_test[cols[-1]] == 1] = '태양광 보유'
df_test[cols[-1]][df_test[cols[-1]] == 0] = '태양광 미보유'

for col in cols:
    df_train = pd.concat([df_train, pd.get_dummies(df_train[col])], axis = 1)
    df_test = pd.concat([df_test, pd.get_dummies(df_test[col])], axis = 1)
    
for col in cols[1:]:
    del df_train[col]
    del df_test[col]   

<ipython-input-140-757461e7f276>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[cols[1]][df_train[cols[1]] == 1] = '냉방설비 운영'
<ipython-input-140-757461e7f276>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[cols[-1]][df_train[cols[-1]] == 1] = '태양광 보유'
<ipython-input-140-757461e7f276>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[cols[1]][df_test[cols[1]] == 1] = '냉방설비 운영'
<ipython-input-140-757461e7f276

In [142]:
df_train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),year,month,...,55,56,57,58,59,60,냉방설비 비운영,냉방설비 운영,태양광 미보유,태양광 보유
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,2020,6,...,0,0,0,0,0,0,1,0,1,0
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,2020,6,...,0,0,0,0,0,0,1,0,1,0
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,2020,6,...,0,0,0,0,0,0,1,0,1,0
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,2020,6,...,0,0,0,0,0,0,1,0,1,0
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,2020,6,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19:00:00,4114.368,27.8,2.3,68.0,0.0,0.7,2020,8,...,0,0,0,0,0,1,0,1,0,1
122396,60,2020-08-24 20:00:00,3975.696,27.3,1.2,71.0,0.0,0.0,2020,8,...,0,0,0,0,0,1,0,1,0,1
122397,60,2020-08-24 21:00:00,3572.208,27.3,1.8,71.0,0.0,0.0,2020,8,...,0,0,0,0,0,1,0,1,0,1
122398,60,2020-08-24 22:00:00,3299.184,27.1,1.8,74.0,0.0,0.0,2020,8,...,0,0,0,0,0,1,0,1,0,1


In [144]:
feature_cols = ['기온(°C)','풍속(m/s)','습도(%)','일조(hr, 3시간)','강수량(mm, 6시간)']

In [145]:
feature_cols = ['기온(°C)','풍속(m/s)','습도(%)','일조(hr, 3시간)','강수량(mm, 6시간)']

for i in range(4):
    for x in range(0,len(df_test),3):
        for j in range(1,3):
            df_test[feature_cols[i]][x+j] = df_test[feature_cols[i]][x]

for x in range(0,len(df_test),6):
    for k in range(1,6):
        df_test['강수량(mm, 6시간)'][x+k] = df_test['강수량(mm, 6시간)'][x]


<ipython-input-145-d34f0d03d57f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[feature_cols[i]][x+j] = df_test[feature_cols[i]][x]
<ipython-input-145-d34f0d03d57f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['강수량(mm, 6시간)'][x+k] = df_test['강수량(mm, 6시간)'][x]


In [148]:
# onehotencoding 

feature_train = df_train.columns.difference(['전력사용량(kWh)','date_time','num'])
feature_test = df_test.columns.difference(['date_time','num'])
x_train = df_train[feature_train]
y_train = df_train[['num','date_time','전력사용량(kWh)']]
x_test = df_test[feature_test]


In [150]:
x_train.shape, y_train.shape, x_test.shape

((122400, 73), (122400, 3), (10080, 73))

In [168]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor, LGBMRanker, LGBMClassifier

LGBMRanker()

In [152]:
# rf = RandomForestRegressor().fit(x_train, y_train['전력사용량(kWh)'])
# y_pred = rf.predict(x_test)
# # rf.score(x_train, y_train['전력사용량(kWh)'])


In [159]:
MLA = [
    # Linear
    LinearRegression(),

    #Ensemble Methods
    #AdaBoostRegressor(),
    BaggingRegressor(),
    ExtraTreesRegressor(),
    GradientBoostingRegressor(),
    RandomForestRegressor(),

    #xgboost
    XGBRegressor(),

    #LGBM
    LGBMRegressor()

]

In [154]:
# for model in MLA :
#     print(str(model).split('(')[0])

In [155]:
x_test

,1,2,3,4,5,6,7,8,9,10,...,year,"강수량(mm, 6시간)",기온(°C),냉방설비 비운영,냉방설비 운영,습도(%),"일조(hr, 3시간)",태양광 미보유,태양광 보유,풍속(m/s)
0,1,0,0,0,0,0,0,0,0,0,...,2020,0.0,27.8,1,0,74.0,0.0,1,0,1.5
1,1,0,0,0,0,0,0,0,0,0,...,2020,0.0,27.8,1,0,74.0,0.0,1,0,1.5
2,1,0,0,0,0,0,0,0,0,0,...,2020,0.0,27.8,1,0,74.0,0.0,1,0,1.5
3,1,0,0,0,0,0,0,0,0,0,...,2020,0.0,27.3,1,0,78.0,0.0,1,0,1.1
4,1,0,0,0,0,0,0,0,0,0,...,2020,0.0,27.3,1,0,78.0,0.0,1,0,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,0,0,0,0,0,0,0,0,0,0,...,2020,0.0,29.0,1,0,65.0,0.8,1,0,3.3
10076,0,0,0,0,0,0,0,0,0,0,...,2020,0.0,29.0,1,0,65.0,0.8,1,0,3.3
10077,0,0,0,0,0,0,0,0,0,0,...,2020,0.0,27.9,0,1,68.0,0.0,0,1,4.1
10078,0,0,0,0,0,0,0,0,0,0,...,2020,0.0,27.9,1,0,68.0,0.0,1,0,4.1


In [156]:
model = AdaBoostRegressor().fit(x_train, y_train['전력사용량(kWh)'])
result = model.predict(x_test)
for model in MLA :
    model.fit(x_train, y_train['전력사용량(kWh)'])
    y_pred = model.predict(x_test)
    result += y_pred

y_pred = result / 8
sub['answer'] = y_pred
sub.to_csv('result_hp_tuning.csv',index = False)

In [162]:
lgbm = LGBMRegressor().fit(x_train, y_train['전력사용량(kWh)'])
y_pred = lgbm.predict(x_test)
sub['answer'] = y_pred
sub.to_csv('result_hp_tuning.csv',index = False)

In [172]:
lgbm = LGBMRegressor().fit(x_train, y_train['전력사용량(kWh)'])
y_pred = lgbm.predict(x_test)
sub['answer'] = y_pred
sub.to_csv('result_hp_tuning.csv',index = False)

In [163]:
y_pred

array([8442.00874614, 8402.79858946, 8397.61848777, ..., 3463.6223268 ,
       3005.8535496 , 3005.8535496 ])

In [176]:
from sklearn.model_selection import GridSearchCV

In [185]:
lgbm_grid = [{'learning_rate' : [ 0.05, 0.1],
         'n_estimators' : [100, 300, 500]}
         ]

In [186]:
lgbm = LGBMRegressor()

In [190]:
lgbm_model = GridSearchCV(lgbm, param_grid= lgbm_grid, cv = 3)
lgbm_model.fit(x_train, y_train['전력사용량(kWh)'])

GridSearchCV(cv=3, estimator=LGBMRegressor(),
             param_grid=[{'learning_rate': [0.05, 0.1],
                          'n_estimators': [100, 300, 500]}])

In [194]:
y_pred = lgbm_model.predict(x_test)

In [195]:
sub['answer'] = y_pred
sub.to_csv('result_hp_tuning.csv',index = False)